In [1]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [3]:
from utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

In [4]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [5]:
def scanner(MODEL_FOLDER,image_path,thresh,label_list):
    
    CWD_PATH = os.getcwd()
    #MODEL_FOLDER = "faster_rcnn_inception_v2_coco_2018_01_28"

    # Path to frozen detection graph .pb file, which contains the model that is used for object detection
    PATH_TO_CKPT = os.path.join(CWD_PATH, MODEL_FOLDER, "frozen_inference_graph.pb")

    # Path to label map file
    PATH_TO_LABELS = os.path.join(CWD_PATH,MODEL_FOLDER, 'mscoco_label_map.pbtxt')

    # Number of classes the object detector can identify
    NUM_CLASSES = 90

    #define threshold of detection
    #thresh= 0.5

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories) 
    
    d=[category_index[i]['id'] for i in range(1,91) if i!=12 and i!=26 and i!=29 and i!=30 and i!=45 and i!=66 and i!=68 and i!=69 and i!=71 and i!=83 and category_index[i]['name'] in label_list]

    # Load the Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)


    # Define input and output tensors (i.e. data) for the object detection classifier

    # Input tensor is the image
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Output tensors are the detection boxes, scores, and classes
    # Each box represents a part of the image where a particular object was detected
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represents level of confidence for each of the objects.
    # The score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Number of objects detected
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    # Each score represents level of confidence for each of the objects.
    # The score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Number of objects detected
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    image_np = np.array(Image.open(image_path))
    img=image_np
    
    items = []
    coordinates = []
        #if you want to resize to tune inference
        #img = cv2.resize(img_org, (300,300))
    img_expanded = np.expand_dims(img, axis=0)
        #print(img_expanded.shape)
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: img_expanded})

    objects = []
    count=0
    for index, value in enumerate(classes[0]):
        object_dict = {}
        if scores[0, index] > thresh:
            object_dict[(category_index.get(value)).get('name')] = scores[0, index]
            objects.append(object_dict)
            count+=1
            #print (objects)
        
    for i in range(count):
        if classes[0][i] not in d:
            classes[0][i]=1
            boxes[0][i]=0
            scores[0][i]=0

        #Get all the detected class labels in one list
    for y in objects:
        for keys in y.keys():
            m = list(y.keys())[0]
            if m in label_list:
                items.append(m)
        
        

    #Get co ordinates of the detected classes
    coordinates = vis_util.return_coordinates(
                    img,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=10,
                    min_score_thresh=thresh)
    vis_util.visualize_boxes_and_labels_on_image_array(
                    img,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=thresh)

    return coordinates,items,img


In [6]:
import sys,os
from PyQt4 import QtGui,QtCore
import xml.etree.cElementTree as xml
import cv2

In [7]:
class Window(QtGui.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(50,50,1550,750)
        self.idx=-1
        self.image_paths=""
        self.alldirs=[]
        self.currlabel=None
        self.setWindowTitle("Auto Labelling Tool")
        self.mylabel=None
        self.coordinates=[]
        self.labels=[]
        self.openfolder=QtGui.QPushButton("Open Folder",self)
        self.nextimg=QtGui.QPushButton("Next Image",self)
        self.previmg=QtGui.QPushButton("Previous Image",self)
        self.saveann=QtGui.QPushButton("Save Annotation",self)
        
        self.c1=QtGui.QCheckBox(self)
        self.c2=QtGui.QCheckBox(self)
        self.c3=QtGui.QCheckBox(self)
        self.c4=QtGui.QCheckBox(self)
        self.c5=QtGui.QCheckBox(self)
        
        self.lf=QtGui.QLabel(self)
        self.sp=QtGui.QDoubleSpinBox(self)
        self.th=QtGui.QLabel(self)
        self.layout = QtGui.QVBoxLayout()
        self.b1 = QtGui.QRadioButton("frcnn",self)
        self.b2 = QtGui.QRadioButton("ssd mobile",self)
        self.b3=QtGui.QRadioButton("ssd inception",self)
        self.detect=QtGui.QPushButton("DETECT",self)
        
        self.home()
        self.show()
        
    
    def home(self):
        
        
        self.openfolder.resize(300,100)
        self.openfolder.move(30,30)
        
        self.nextimg.resize(300,100)
        self.nextimg.move(30,200)
        
        self.previmg.resize(300,100)
        self.previmg.move(30,370)
        
        self.saveann.resize(300,100)
        self.saveann.move(30,540)
        self.l1=QtGui.QLabel(self)
        self.l1.setText("SELECT MODEL")
        self.l1.resize(200,30)
        #self.l1.resize(300,300)
        self.l1.move(self.width()-200,0)
        self.openfolder.clicked.connect(self.OpenFolderClicked)
        self.nextimg.clicked.connect(self.NextImageClicked)
        self.previmg.clicked.connect(self.PreviousImageClicked)
        
        
        
        self.th.setText("DETECTION THRESHOLD")
        self.th.resize(200,30)
        self.th.move(self.width()-200,200)
        
        
        self.sp.move(self.width()-200,250)
        self.sp.resize(200,30)
        self.sp.setRange(0,1)
        self.sp.setSingleStep(0.05)
        
        #Labels
        
        self.lf.setText("LABEL FILTER")
        self.lf.resize(200,50)
        self.lf.move(self.width()-200,300)
        
        self.c1.setText("person")
        
        self.c2.setText("cat")
        
        self.c3.setText("dog")
        
        self.c4.setText("bottle")
        
        self.c5.setText("chair")
        
        self.c1.resize(200,30)
        self.c2.resize(200,30)
        self.c3.resize(200,30)
        self.c4.resize(200,30)
        self.c5.resize(200,30)
        
        self.c1.move(self.width()-200,350)
        self.c2.move(self.width()-200,380)
        self.c3.move(self.width()-200,410)
        self.c4.move(self.width()-200,440)
        self.c5.move(self.width()-200,470)
        
        #box layout
        
        self.layout.stretch(10)
        
        self.layout.addWidget(self.b1)
        
        self.layout.addWidget(self.b2)
        
        self.layout.addWidget(self.b3)
        self.b1.move(self.width()-200,50)
        self.b2.move(self.width()-200,100)
        self.b3.move(self.width()-200,150)
        self.b1.resize(150,30)
        self.b2.resize(150,30)
        self.b3.resize(150,30)
        
        
        
        self.detect.resize(200,100)
        self.detect.move(self.width()-200,550)
        self.detect.clicked.connect(self.detectclicked)
        self.saveann.clicked.connect(self.saveannclicked)
    
    def saveannclicked(self):
        if(len(self.coordinates)==0 and len(self.labels)==0):
            return
        filename="C:\\Users\\Bhagesh\\Desktop\\"+self.alldirs[self.idx][:-4]+".xml"
        root=xml.Element("annotation")
        folder=xml.SubElement(root,"folder")
        folder.text=self.alldirs[self.idx]
        fname=xml.SubElement(root,"filename")
        fname.text=self.alldirs[self.idx]
        path=xml.SubElement(root,"path")
        mypath=self.image_paths+"\\"+self.alldirs[self.idx]
        path.text=mypath
        img=cv2.imread(mypath)
        size=xml.Element("size")
        width=xml.SubElement(size,"width")
        width.text=str(img.shape[1])
        height=xml.SubElement(size,"height")
        height.text=str(img.shape[0])
        root.append(size)
        
        for i in range(len(self.coordinates)):
            obj=xml.Element("object")
            name=xml.SubElement(obj,"name")
            name.text=self.labels[i]
            bndbox=xml.Element("bndbox")
            xmin=xml.SubElement(bndbox,"xmin")
            xmin.text=str(self.coordinates[i][2])
            xmax=xml.SubElement(bndbox,"xmax")
            xmax.text=str(self.coordinates[i][3])
            ymin=xml.SubElement(bndbox,"ymin")
            ymin.text=str(self.coordinates[i][0])
            ymax=xml.SubElement(bndbox,"ymax")
            ymax.text=str(self.coordinates[i][1])
            obj.append(bndbox)
            root.append(obj)
        
        with open(filename,"w") as f:
            finalfile=xml.tostring(root,encoding='unicode')
            f.write(finalfile)
        
        f.close()
        
        
    
    def renderimage(self):
        self.deleteimage()
        self.currlabel=QtGui.QLabel(self)
        mypath=self.image_paths+"\\"+self.alldirs[self.idx]
        pixmap=QtGui.QPixmap(mypath)
        self.currlabel.setPixmap(pixmap)
        self.currlabel.resize(900,self.height())
        self.currlabel.move(400,0)
        QtGui.QLabel.show(self.currlabel)
    
    def OpenFolderClicked(self):
        self.image_paths = str(QtGui.QFileDialog.getExistingDirectory(self, "Select Directory"))
        self.idx=0
        self.alldirs=os.listdir(self.image_paths)
        self.renderimage()
        
    
    def NextImageClicked(self):
        if self.idx==len(self.alldirs)-1:
            return
        else:
            self.idx+=1
            self.renderimage()
    
    def PreviousImageClicked(self):
        if(self.idx==0):
            return
        else:
            self.idx-=1;
            self.renderimage()
            
    def deleteimage(self):
        if(self.currlabel!=None):
            self.currlabel.resize(0,0)
        if(self.mylabel!=None):
            self.mylabel.resize(0,0)
    
    
    def detectclicked(self):
        labels=[]
        model=""
        thresh=None
        thresh=self.sp.value()
        if self.b1.isChecked() is True:
            model="faster_rcnn_inception_v2_coco_2018_01_28"
        elif self.b2.isChecked() is True:
            model="ssd_mobilenet_v1_coco_2018_01_28"
        elif self.b3.isChecked() is True:
            model="ssd_inception_v2_coco_2018_01_28"
        
        if self.c1.isChecked() is True:
            labels.append(self.c1.text())
        if self.c2.isChecked() is True:
            labels.append(self.c2.text())
        if self.c3.isChecked() is True:
            labels.append(self.c3.text())
        if self.c4.isChecked() is True:
            labels.append(self.c4.text())
        if self.c5.isChecked() is True:
            labels.append(self.c5.text())
        
        mypath=self.image_paths+"\\"+self.alldirs[self.idx]
        if(model=="" or len(labels)==0 or thresh==None):
            return;
        c,l,img=scanner(model,mypath,thresh,labels)
        self.coordinates=c
        self.labels=l
        height, width, channel = img.shape
        bytesPerLine = 3 * width
        qImg = QtGui.QImage(img.data, width, height, bytesPerLine, QtGui.QImage.Format_RGB888)
        self.mylabel=QtGui.QLabel(self)
        self.mylabel.setPixmap(QtGui.QPixmap.fromImage(qImg))
        self.deleteimage()
        self.mylabel.resize(900,self.height())
        self.mylabel.move(400,0)
        QtGui.QLabel.show(self.mylabel)

def run():
    app=QtGui.QApplication(sys.argv)
    GUI=Window()
    sys.exit(app.exec_())



In [8]:
run()

SystemExit: 0

c:\users\bhagesh\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
